In [1]:
import azureml
import json

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment
from azureml.core.dataset import Dataset

import tensorflow as tf
from IPython.display import Image, display

import matplotlib.pyplot as plt
import tensorflow_hub as hub

import numpy as np

import warnings
warnings.filterwarnings('ignore')

2022-09-09 19:29:12.309559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="d6233897-5c9f-47f9-8507-6d4ada2d5183")
resource_group = os.getenv("RESOURCE_GROUP", default="RG_Jaganlal")
workspace_name = os.getenv("WORKSPACE_NAME", default="beanleaf_classifier")
workspace_region = os.getenv("WORKSPACE_REGION", default="centralus")
cluster_name = os.getenv("CLUSTER_NAME", default="beanleaf-classifier-cluster")
dataset_path_in_datastore = os.getenv("BLOB_STORAGE_PATH", default="beanleaf")
data_local_directory = os.getenv("DATA_LOCAL_DIRECTORY", default="./beans/")
experiment_name = os.getenv("EXPERIMENT_NAME", default="BeanClassifierExperiment")
blob_container = os.getenv("BLOB_CONTAINER", default="beanleaf_dataset")

In [3]:
ws = Workspace.from_config()
ws

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace.create(name='beanleaf_classifier', subscription_id='d6233897-5c9f-47f9-8507-6d4ada2d5183', resource_group='RG_Jaganlal')

In [18]:
from azureml.core import ComputeTarget
target = ComputeTarget(ws, cluster_name)
target

AmlCompute(workspace=Workspace.create(name='beanleaf_classifier', subscription_id='d6233897-5c9f-47f9-8507-6d4ada2d5183', resource_group='RG_Jaganlal'), name=beanleaf-classifier-cluster, id=/subscriptions/d6233897-5c9f-47f9-8507-6d4ada2d5183/resourceGroups/RG_Jaganlal/providers/Microsoft.MachineLearningServices/workspaces/beanleaf_classifier/computes/beanleaf-classifier-cluster, type=AmlCompute, provisioning_state=Succeeded, location=centralus, tags={})

# Connect to Workspace

In [19]:
# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print("Compute Target:", compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:", ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name=experiment_name)
print("Experiment:", exp.name)

Compute Target: beanleaf-classifier-cluster
Datastore: workspaceblobstore
Experiment: BeanClassifierExperiment


In [ ]:
from azureml.data.datapath import DataPath

# Upload data to AzureML Datastore
ds = ws.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=data_local_directory,
            target=DataPath(ds, 'beanleaf_dataset'),
            show_progress=True, overwrite=True)

# Register file dataset with AzureML
ds = ds.register(workspace=ws, name="beanleaf", description="Bean leaves with train, test and validation folders", create_new_version=True)
print(f'Dataset {ds.name} registered.')

In [99]:
datastore_paths = [(ds, os.path.join('beanleaf_dataset', 'train/healthy/healthy_train.0.jpg'))]
data_ds = Dataset.File.from_files(path=datastore_paths)
data_ds

{
  "source": [
    "('workspaceblobstore', 'beanleaf_dataset/train/healthy/healthy_train.0.jpg')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

In [76]:
input_data = Dataset.File.from_files(ds.path('beanleaf_dataset')).as_named_input('input').as_mount()

In [80]:
%%writefile ./dummy_train.py

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import sys
import os
from os.path import exists
import tensorflow as tf

print("*********************************************************")
print("Hello Azure ML!")

mounted_input_path = sys.argv[1]
# mounted_output_path = sys.argv[2]

print("Argument 1: %s" % mounted_input_path)

path_to_file = os.path.join(mounted_input_path, 'train/healthy/healthy_train.0.jpg')
print("Path to file:", path_to_file)
file_exists = exists(path_to_file)
print("File healthy_train.0.jpg - ", file_exists)

batch_size = 128
img_height = 224
img_width = 224

training_path = os.path.join(mounted_input_path, 'train')
training_path_exists = exists(training_path)
print("training_path_exists - ", training_path_exists)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(training_path,
                                                        seed=111,
                                                        image_size=(img_height, img_width),
                                                        batch_size=batch_size)

print('Training DS')

Overwriting ./dummy_train.py


In [61]:
# dataset = Dataset.get_by_name(ws, name='beanleaf', version='latest')
# dataset_folder = dataset.as_mount()

In [65]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk>=1.12.0', 'tensorflow'])

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [81]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = './',
                      script = 'dummy_train.py',
                      arguments = [input_data],
                      compute_target = compute_target,
                      environment = myenv)
src


In [89]:
run = exp.submit(config=src)

In [83]:
run.wait_for_completion(show_output=True)

RunId: MountCheck_1662778848_dc70823a
Web View: https://ml.azure.com/runs/MountCheck_1662778848_dc70823a?wsid=/subscriptions/d6233897-5c9f-47f9-8507-6d4ada2d5183/resourcegroups/RG_Jaganlal/workspaces/beanleaf_classifier&tid=95917aa5-d840-4443-a19e-aa3ad09d4bb5

Streaming user_logs/std_log.txt

2022-09-10 03:04:16.337403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-10 03:04:16.443568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_08611e4a6005c389e3f64e208f5064d3/lib:
2022-09-10 03:04:16.443598: I tensorflow/stream_executo

{'runId': 'MountCheck_1662778848_dc70823a',
 'target': 'beanleaf-classifier-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-10T03:03:08.853655Z',
 'endTimeUtc': '2022-09-10T03:04:21.753625Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': 'ce0daa73-b7e6-406a-ad7c-965cd78f3395',
  'azureml.git.repository_uri': 'git@github.com-jaganlal:jaganlal/beanleaf-disease-classifier.git',
  'mlflow.source.git.repoURL': 'git@github.com-jaganlal:jaganlal/beanleaf-disease-classifier.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'ae8af000d23713c4cce213e7a1880a2302fe4c36',
  'mlflow.source.git.commit': 'ae8af000d23713c4cce213e7a1880a2302fe4c36',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'de256d06-8a1a-491e-910c-e63da3735e95'}, 'consumptionDetails':

In [ ]:
ds1 = ws.get_default_datastore()
ds1.download('./')

In [ ]:
!find beans -type f | wc -l

In [ ]:
!find beans/test -type f | wc -l

In [ ]:
!find beans/validation -type f | wc -l

In [8]:
display(Image(dataset_folder+'/beanleaf_dataset/train/healthy/healthy_train.0.jpg'))

TypeError: unsupported operand type(s) for +: 'DatasetConsumptionConfig' and 'str'

In [ ]:
display(Image('beans/train/angular_leaf_spot/angular_leaf_spot_train.124.jpg'))

In [ ]:
display(Image('beans/train/bean_rust/bean_rust_train.162.jpg'))

In [ ]:
batch_size = 128
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('beans/train',
  seed=111,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory('beans/test',
  seed=111,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory('beans/validation',
  seed=111,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
classes=train_ds.class_names
print(classes)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(classes[labels[i]])
    plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
feature_extractor = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor, input_shape=(img_height,img_width,3))

In [ ]:
feature_extractor_layer.trainable = False

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
tf.random.set_seed(111)

In [ ]:
model = tf.keras.Sequential([
  normalization_layer,
  feature_extractor_layer,
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(3,activation='softmax')
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=20, validation_data=val_ds)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'val_acc'], loc='best')
plt.show()

In [ ]:
result=model.evaluate(test_ds)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in test_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)

    plt.tight_layout()
    
    img = tf.keras.preprocessing.image.img_to_array(images[i])                    
    img = np.expand_dims(img, axis=0)  

    pred=model.predict(img)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title("Actual Label: %s" % classes[labels[i]])
    plt.text(1, 240, "Predicted Label: %s" % classes[np.argmax(pred)], fontsize=12)

    plt.axis("off")

In [ ]:
model.save('./models', save_format='tf')

In [ ]:
!ls -alrt models

In [ ]:
model_loaded = tf.keras.models.load_model('./models/')

In [ ]:
model_loaded.summary()

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def process(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [ ]:
pred_label=model_loaded.predict(process('beans/train/healthy/healthy_train.0.jpg'))
print(classes[np.argmax(pred_label)])

In [ ]:
pred_label